These code snippets generate the dataframe images used in the README file. The first snippet generates the representative data (10 rows from the reduced model) from a CSV file that was reformatted in OpenOffice to make it more understandable. The second snippet generates an image of the coefficient dataframe that displays the coefficient estimates for the reduced model. In both cases, the dataframe_image module.

In [1]:
import pandas as pd
import numpy as np
import dataframe_image as dfi

In [2]:
# Export an image of representative data (10 rows)

heart_model_input_df = pd.read_csv('../Data/heart_model_input.csv')

In [3]:
# Make a data frame with representative values for presentation

# Drop unneccessary columns
# Note were are using the reduced model for this

drop_list = ['rest_bp', 'chol', 'blood_sugar', 'restecg', 'age_scaled', 'rest_bp_scaled', 'chol_scaled', 'max_heart_rate_scaled', 'oldpeak_scaled', 'outlier']
heart_model_input_df.drop(drop_list, axis='columns', inplace=True)

In [7]:
# Create a df with a small random sample of the data
# so the Boolean fields aren't all one value
# You may need to run it a few times to get a good sample

random_state = 321

# Make a new df, since we need the full one later

heart_model_sample_df = heart_model_input_df.sample(n = 10, random_state = random_state, replace = False)
heart_model_sample_df.head(10)

,age,sex,chest_pain,max_heart_rate,angina,oldpeak,ST_slope,flouroscopy,heart_defect,heart_disease
35,46,0,1,160,1,1.4,1,0,0,0
133,49,0,0,163,0,0.0,1,0,0,0
101,42,1,1,194,0,0.8,1,0,1,0
59,71,0,1,130,0,0.0,1,1,0,0
249,45,1,0,147,1,0.0,0,1,1,1
118,64,0,0,122,0,2.0,0,1,0,0
163,63,1,0,147,0,1.4,0,1,1,1
107,50,0,0,159,0,0.0,1,0,0,0
196,60,1,0,141,1,2.8,0,1,1,1
156,56,1,1,163,0,0.0,1,0,1,0


In [8]:
# Change Boolean features to text and recode them
# with more understandable names

# Make a dictionary with each column dtype as string
# Then do the cast

cast_list = ['sex', 'chest_pain', 'angina', 'ST_slope', 'flouroscopy', 'heart_defect', 'heart_disease']
cast_dict = {k: 'string' for k in cast_list}
heart_model_cast_df = heart_model_sample_df.astype(cast_dict).copy()

# Now, do the recode
# Sex and flouroscopy have different values and are handled separately

heart_model_cast_df.replace({'sex': {'0': 'Male', '1': 'Female'}}, inplace = True)
heart_model_cast_df.replace({'flouroscopy': {'0': 'Zero', '1': 'Nonzero'}}, inplace = True)

recode_list = ['chest_pain', 'angina', 'ST_slope', 'heart_defect', 'heart_disease']
recode_dict = {k: {'0': 'No', '1': 'Yes'} for k in recode_list}

heart_model_cast_df.replace(recode_dict, inplace=True)
heart_model_cast_df.reset_index(drop=True, inplace=True)

In [9]:
# Generate an image of the df to add to the README

dfi.export(heart_model_cast_df, '../Images/heart_df_present.png')

In [ ]:
# Code to export the coefficient dataframe after making it easier to read.

# Load the full coefficient dataframe

coef_df_all = pd.read_csv('../Data/coefficient_df.csv')
coef_df_all.set_index('coef', inplace = True)

In [ ]:
# Generate the df containing only the reduced model to export

df = coef_df_all.copy()

# Drop uneccesary columns and rows

df.drop(['clf_all', 'clf_trimmed'], axis='columns', inplace=True)
df.drop(['rest_bp', 'chol', 'blood_sugar', 'restecg'], axis='index', inplace=True)

# Rename column and index so they are easier to understand

df.rename_axis(index= {'coef': 'feature'}, axis='index', inplace=True)
df.rename(columns = {'clf_reduced': 'value'}, inplace=True)

# Round feature column to three decimal places

# df_formatted = df.round({'value': 2})
# df_formatted

pd.options.display.float_format = '{:20,.2f}'.format
dfi.export(df, '../Images/coefficient_figure.png')

In [55]:
# Export summary statistics for the reduced model
# Need to set float format so only two decimals are displayed

cols = ['age', 'max_heart_rate', 'oldpeak']

summary_df = heart_model_input_df[cols].describe().copy()
summary_df.loc['count'].apply(pd.to_numeric)
#summary_df.loc['count']

age                            296.00
max_heart_rate                 296.00
oldpeak                        296.00
Name: count, dtype: float64

In [13]:
# Export the coefficient matrix (as a heat map) of the 4 continuous variables
# in the reduced model

# Need to format floats and ints separately
# so ints don't print with decimals

formatter = {
    'int': lambda x: "{0:10.0f}".format(x),
    'float': lambda x: "{0:10.2f}".format(x)
}

np.set_printoptions(linewidth = 300, suppress=True, formatter=formatter)
